In [ ]:
# default_exp splitting

# Splitting

> Simple functions to save time when creating validation and testing splits

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [ ]:
df = pd.DataFrame(np.random.randint(1,101,10000),columns=['dep_var'])
df.head()

,dep_var
0,5
1,69
2,52
3,90
4,25


In [ ]:
#export
def bin_df(df,col,bin_sz):
    col_range = (df[col].max() - df[col].min() + 1)
    if not col_range % bin_sz == 0:
        for new_sz in range(bin_sz,df[col].max()):
            if col_range % new_sz == 0:
                print(f'bin sizes not right - next largest size is {new_sz}')
                assert col_range % bin_sz == 0
    for i in range(df[col].min(),df[col].max(),bin_sz):
        bin_min = i
        bin_max = i + bin_sz - 1
        mask = (df[col] >= bin_min) & (df[col] <= bin_max)
        df.loc[mask,f'bin_{bin_sz}'] = f'{bin_min}-{bin_max}'
    return df

In [ ]:
df = bin_df(df,'dep_var',25)
df.head()

,dep_var,bin_25,3fold_bin_25
0,5,1-25,0.0
1,69,51-75,2.0
2,52,51-75,1.0
3,90,76-100,2.0
4,25,1-25,0.0


In [ ]:
#export
def kfold_Stratified_df(df,col,folds,shuffle=True):
    skf = StratifiedKFold(n_splits=folds,shuffle=shuffle)
    col_name = f'{folds}fold_{col}'
    f_num = 0
    for train_idxs, valid_idxs in skf.split(df.index,df[col]):
        df.loc[df.index.isin(valid_idxs),col_name] = int(f_num)
        f_num += 1
    return df

In [ ]:
df = kfold_Stratified_df(df,'bin_25',3)
df.groupby('3fold_bin_25').count()

,dep_var,bin_25
3fold_bin_25,,
0.0,3334,3334
1.0,3333,3333
2.0,3333,3333


In [ ]:
df.groupby(['3fold_bin_25','bin_25']).count().reset_index().sort_values(['bin_25','3fold_bin_25'])

,3fold_bin_25,bin_25,dep_var
0,0.0,1-25,862
4,1.0,1-25,862
8,2.0,1-25,862
1,0.0,26-50,822
5,1.0,26-50,821
9,2.0,26-50,822
2,0.0,51-75,814
6,1.0,51-75,814
10,2.0,51-75,813
3,0.0,76-100,836
